In [ ]:
# Parameters : 入力パラメータ
RR: str = ""
"""The filepath of RR data to analyze."""
RL: str = ""
"""The filepath of RL data to analyze."""
outputdir: str = "."
"""Path to the directory to save the csv files of the analyzed data."""
wavelength_range: list[float] | None = None
"""The wavelength range to find peaks."""
dump_csv: bool = True
"""Whether the dataframe of analyzed data is saved as CSV files.""";

In [ ]:
# Validate parameters : 入力パラメータを検証
import pathlib

if not pathlib.Path(RR).exists():
    raise FileNotFoundError(f"{RR} not found")
if not pathlib.Path(RL).exists():
    raise FileNotFoundError(f"{RL} not found")
if not (path := pathlib.Path(outputdir)).exists():
    path.mkdir(parents=True)
if (wavelength_range is not None) and (len(wavelength_range) != 2):
    raise ValueError("The length of `wavelength_range` must be 2")

In [ ]:
# Define data types : データ型を定義
import os
import typing as t

import pandas as pd

T = t.TypeVar("T")
Path = t.TypeVar("Path", bound=os.PathLike[str])


class DataPair(t.NamedTuple, t.Generic[T]):
    RR: T
    RL: T


class AnalyzedData(t.NamedTuple):
    wdf: DataPair[pd.DataFrame]
    tdf: DataPair[pd.DataFrame]

In [ ]:
# Defilne `analyze` function : データを解析する関数を定義
import numpy as np
from tlab_analysis import apparatus, trpl, utils


def analyze(filepath: DataPair[str] | DataPair[Path]) -> AnalyzedData:
    # データの読み込み
    data = DataPair(*map(trpl.read_file, filepath))
    # 感度補正
    data = DataPair(*map(apparatus.streakscope.correct, data))
    # ピークを取得
    wdf = DataPair(*map(lambda d: d.aggregate_along_time(), data))
    range_to_find_peak = tuple(
        wavelength_range or wdf.RR["wavelength"].quantile([0.20, 0.80])
    )
    is_to_find_peak = wdf.RR["wavelength"].between(*range_to_find_peak)
    peaks = utils.find_peaks(
        wdf.RR["wavelength"][is_to_find_peak],
        wdf.RR["intensity"][is_to_find_peak],
    )
    # FWHM範囲を取得
    mean_wavelength = wdf.RR["wavelength"].mean()
    peaks.sort(key=lambda peak: abs(peak.x - mean_wavelength))
    FWHM_range = (peaks[0].x0, peaks[0].x1) if len(peaks) > 0 else range_to_find_peak
    tdf = DataPair(*map(lambda d: d.aggregate_along_wavelength(FWHM_range), data))
    for _tdf in tdf:
        _tdf["intensity"] = utils.smooth(_tdf["intensity"], 0.025)
    return AnalyzedData(wdf, tdf)

In [ ]:
# Define `plot` function : データをプロットする関数を定義
import plotly.graph_objects as go
import plotly.io

plotly.io.renderers.default = "notebook+svg"


def plot(data: AnalyzedData) -> go.Figure:
    fig = (
        go.Figure()
        .set_subplots(rows=1, cols=2)
        .update_layout(
            width=960,
            height=480,
            legend1=go.layout.Legend(
                xanchor="right",
                x=0.45,
                yanchor="top",
                y=1.0,
            ),
            legend2=go.layout.Legend(
                xanchor="right",
                x=1.0,
                yanchor="top",
                y=1.0,
            ),
        )
    )
    line_name = DataPair("RR", "RL")
    line_color = DataPair("red", "blue")
    # スペクトル図を作成
    for i, wdf in enumerate(data.wdf):
        fig.add_scatter(
            x=wdf["wavelength"],
            y=wdf["intensity"],
            line=go.scatter.Line(color=line_color[i]),
            name=line_name[i],
            showlegend=True,
            legend="legend1",
            row=1,
            col=1,
        )
    FWHM_range = data.tdf[0].attrs["wavelength_range"]
    fig.add_vrect(
        *FWHM_range,
        line=go.layout.shape.Line(width=0),
        fillcolor="orange",
        opacity=0.2,
        showlegend=False,
        annotation=go.layout.Annotation(
            text=f"FWHM: {FWHM_range[1] - FWHM_range[0]:.1f}nm"
        ),
        row=1,
        col=1,
    ).update_xaxes(
        title="Wavelength (nm)",
        row=1,
        col=1,
    ).update_yaxes(
        title="Intensity (arb.units)",
        row=1,
        col=1,
    )
    # PL強度の時間変化グラフを作成
    for i, tdf in enumerate(data.tdf):
        fig.add_scatter(
            x=tdf["time"],
            y=tdf["intensity"],
            line=go.scatter.Line(color=line_color[i]),
            name=line_name[i],
            showlegend=True,
            legend="legend2",
            row=1,
            col=2,
        )
    fig.update_xaxes(
        title="Time (ns)",
        row=1,
        col=2,
    ).update_yaxes(
        title="Intensity (arb.units)",
        row=1,
        col=2,
    )
    return fig

In [ ]:
# Analyze each data : それぞれのデータを解析
import difflib

filepath = DataPair(*map(pathlib.Path, (RR, RL)))
match = difflib.SequenceMatcher(
    a=filepath.RR.stem,
    b=filepath.RL.stem,
).find_longest_match()
filename = filepath.RR.stem[match.a : match.a + match.size]
analyzed = analyze(filepath)
fig = plot(analyzed)
fig.show()

In [ ]:
# Defilne `analyze_spin_polarization` function : スピン偏極率を解析する関数を定義


def analyze_spin_polarization(
    data: AnalyzedData,
    fitting_function: t.Any,
    shift: int = 0,
    **kwargs: t.Any,
) -> pd.DataFrame:
    df = pd.DataFrame(
        dict(
            time=data.tdf.RR["time"],
            RR=data.tdf.RR["intensity"],
            RL=data.tdf.RL["intensity"].shift(
                int(
                    (data.tdf.RL["time"] >= 0).argmax()
                    - (data.tdf.RR["time"] >= 0).argmax()
                )
                + shift,
                fill_value=0,
            ),
        )
    )
    df["SP"] = (df["RR"] - df["RL"]) / (df["RR"] + df["RL"]) * 100
    df["SP"] = np.where(
        (df["time"] >= 0) & (df["SP"] >= 0) & (df["SP"] <= 100),
        df["SP"],
        0,
    )
    decay_range = utils.find_decay_range(df["SP"], high=0.9, low=0.3)
    is_decay = np.full(df["SP"].shape, False)
    is_decay[decay_range[0] : decay_range[1]] = True
    try:
        params, _ = utils.curve_fit(
            fitting_function,
            df["time"][is_decay],
            df["SP"][is_decay],
            **kwargs,
        )
        df["fit"] = np.where(is_decay, fitting_function(df["time"], *params), np.nan)
        df.attrs["params"] = params
    except Exception:
        df["fit"] = np.nan
    return df

In [ ]:
# Define `plot_spin_polarization` function : スピン偏極率をプロットする関数を定義


def plot_spin_polarization(df: pd.DataFrame) -> go.Figure:
    fig = (
        go.Figure()
        .set_subplots(rows=1, cols=2)
        .update_layout(
            width=960,
            height=480,
            legend1=go.layout.Legend(
                xanchor="right",
                x=0.45,
                yanchor="top",
                y=1.0,
            ),
            legend2=go.layout.Legend(
                xanchor="right",
                x=1.0,
                yanchor="top",
                y=1.0,
            ),
        )
    )
    line_name = DataPair("RR", "RL")
    line_color = DataPair("red", "blue")
    # PL強度の時間変化グラフを作成
    for name, color in zip(line_name, line_color):
        fig.add_scatter(
            x=df["time"],
            y=df[name],
            line=go.scatter.Line(color=color),
            name=name,
            showlegend=True,
            legend="legend1",
            row=1,
            col=1,
        )
    fig.update_xaxes(
        title="Time (ns)",
        row=1,
        col=1,
    ).update_yaxes(
        title="Intensity (arb.units)",
        row=1,
        col=1,
    )
    # スピン編極率の時間変化グラフを作成
    fig.add_scatter(
        x=df["time"],
        y=df["SP"],
        line=go.scatter.Line(color="red"),
        name="spin polarization",
        showlegend=False,
        row=1,
        col=2,
    ).update_xaxes(
        title="Time (ns)",
        row=1,
        col=2,
    ).update_yaxes(
        title="Spin Polarization (%)",
        type="log",
        range=(0, 2),
        row=1,
        col=2,
    )
    if not df["fit"].isna().all():
        fig.add_scatter(
            x=df["time"],
            y=df["fit"],
            line=go.scatter.Line(color="black"),
            showlegend=False,
            row=1,
            col=2,
        )
    return fig

In [ ]:
# Analyze spin polarization : スピン偏極率を解析
import itertools
import operator


def exponential(t: float, a: float, tau: float) -> float:
    return a * np.exp(-t / tau)  # type: ignore[no-any-return]


func = exponential
for shift in itertools.starmap(operator.mul, itertools.product(range(5), [1, -1])):
    df = analyze_spin_polarization(analyzed, func, shift, bounds=(0, np.inf))
    if "params" in df.attrs:
        for key, value in zip(list(func.__annotations__)[1:-1], df.attrs["params"]):
            print(f"{key} = {value:.4g}")
        break
fig = plot_spin_polarization(df)
fig.show()

In [ ]:
# Dump the results as CSV files : 解析結果をCSVファイルとして出力
if dump_csv:
    df.to_csv(
        os.path.join(
            outputdir,
            "spin({left:d}-{right:d})_".format(
                left=round(analyzed.tdf[0].attrs["wavelength_range"][0]),
                right=round(analyzed.tdf[0].attrs["wavelength_range"][1]),
            )
            + filename
            + ".csv",
        ),
        index=False,
    )